# Parse SQL Categories and Index Terms

In [ ]:
import re
from collections import defaultdict
import pandas as pd
import random

text = '''
    Active Databases: 'Active databases'
    Aggregate Functions: 'Aggregation', 'AVG', 'COUNT', 'MAX', 'MIN', 'SUM'
    Aliases & Correlation: 'aliases', 'table alias', 'correlation name', 'correlation variables', 'tuple variables', 'lateral clause', 'AS'
    Arity: 'arity'
    Atomicity & Domains: 'atomic domains', 'atomicity'
    Authorization & Privileges: 'authorization', 'authorization graph', 'privileges', 'grant command', 'revoke privileges', 'select privilege', 'references privilege', 'roles', 'create role', 'set role', 'row-level authorization', 'sql security invoker', 'passwords', 'security', 'sys.context function', 'superusers', 'VPD (Virtual Private Database)', 'granted by current role', 'execute privilege'
    Backup & Recovery: 'backup'
    Business Logic: 'business logic'
    Common Language Runtime (CLR): 'Common Language Runtime (CLR)'
    Cartesian & Product: 'Cartesian products', 'Product'
    Catalogs & Metadata: 'catalogs'
    Change Tracking & Delta: 'delta relation', 'change relation'
    Cursor Operations: 'fetching', 'updatable result sets', 'next method'
    Data Definition Language (DDL): 'Data definition language (DDL)'
    Data Manipulation Language (DML): 'Data Manipulation Language (DML)', 'Insertion', 'deletion', 'Update', 'change relation', 'tuples'
    Data Types - Large Objects: 'large-object types', 'blobs', 'clobs'
    Data Types - Scalar: 'char', 'varchar', 'nvarchar', 'numeric', 'float', 'real', 'double precision', 'Bit string', 'datetime data type', 'timestamp', 'interval data type'
    Database Systems: 'IBM DB2', 'Microsoft SQL Server', 'MySQL', 'Oracle', 'PostgreSQL', 'Informix UDS', 'System R', 'database-management systems (DBMSs)', 'database instance', 'databases', 'databases administrator (DBA)'
    Difference & EXCEPT: 'minus', 'except all', 'except clause', 'except construct', 'Difference operation', 'set-difference operation'
    Dirty Data: 'Dirty data'
    Domain & Check Constraints: 'domain constraints', 'check constraints', 'check clause', 'default values', 'set default', 'not null', 'Assertions in SQL', 'create assertion', 'add constraint', 'CREATE DOMAIN', 'domain of attributes'
    Duplicate Handling: 'Duplicate elimination', 'Duplicates in SQL'
    Embedded & Dynamic SQL: 'embedded SQL', 'embedded databases', 'dynamic SQL', 'EXEC SQL', 'host language'
    Example Databases: 'banking', 'university database', 'sandbox'
    Exceptions & Debugging: 'exceptions', 'exception conditions', 'sqlstate', 'debugging', 'bugs'
    Expressions & Syntax: 'Expressions in SQL', 'syntax', 'WHERE', 'FROM', 'IN', 'in construct', 'not in construct', 'not exists construct', 'some construct', 'some function', 'EXISTS', 'ANY', 'ALL', 'case construct', 'decode', 'empty relations test'
    Fetch/Result APIs: 'application program interfaces (APIs)', 'Call Level Interface (CLI) standards', 'Open Database Connectivity (ODBC)', 'Generic interface', 'DriverManager class', 'getConnection method', 'Statement object', 'ResultSet object', 'getFloat method', 'getString method', 'ADO.NET', 'try-with-resources construct', 'jdbc (java database connectivity)', 'getcolumncount method'
    Group BY & Having: 'GROUP BY', 'group by clause', 'Grouping in SQL', 'grouping sets construct', 'rollup clause', 'rollup construct', 'HAVING', 'cube construct'
    Hierarchies: 'hierarchies', 'start with/connect by prior syntax'
    Identity Columns: 'identity specification'
    Index: 'create index', 'create unique index', 'drop index'
    Integrity Constraints: 'integrity constraints', 'deferred integrity constraints', 'initially deferred integrity constraints', 'set null'
    Join Operations: 'Join', 'Natural join', 'CROSS JOIN', 'Left outerjoin', 'Right outerjoin', 'inner joins', 'Outer join', 'anti-join operation', 'semijoin operation', 'on condition', 'join using operation', 'full outer join'
    Key Constraints: 'keys', 'candidate keys', 'primary keys', 'superkeys', 'unique construct', 'unique key values', 'not unique construct'
    Language Integrated Query (LINQ): 'Language Integrated Query (LINQ)'
    Logical Connectives: 'and connective', 'or connective', 'not connective', 'not operation', 'Negation', 'Boolean operations', 'or operation'
    Null & Unknown Handling: 'Null value', 'UNKNOWN', 'unknown values', 'is null', 'is not null', 'is unknown', 'is not unknown', 'Three-valued logic', 'Truth value', 'true predicate', 'true values', 'false values'
    Operating Systems: 'Unix'
    Ordering & Limits: 'ORDER BY', 'asc expression', 'desc expression', 'limit clause', 'Lexicographic order'
    Partitioning: 'partitions'
    Pointers: 'pointers'
    Prepared Statements: 'prepared statements', 'parameter style general', 'call statement', 'parameterized views'
    Procedures & PSM: 'procedures', 'create procedure', 'functions', 'create function', 'handlers', 'procedural languages', 'Persistent Storage Module (PSM)', 'PL/SQL', 'begin atomic...end', 'repeat loop', 'repeat statements', 'while loop', 'while statements', 'if clauses', 'if-then-else statements', 'then clause', 'when clause', 'when statement', 'nondeclarative actions', 'Packages in SQL: 1999', 'declare statement', 'iteration', 'external language routines'
    Programming Languages: 'C', 'C++', 'Java', 'Perl', 'Python', 'Tcl', 'Visual Basic', 'TransactSQL', 'programming languages'
    Projection & Project Operation: 'project operation', 'Projection', 'Attribute'
    Queries & Paradigms: 'queries', 'query languages', 'declarative queries', 'functional query language', 'imperative query language'
    Recursive Queries: 'recursive queries', 'with recursive clause', 'fixed point of recursive view definition', 'transitive closure'
    Referential Integrity: 'referential integrity', 'references', 'referenced relation', 'referencing relation', 'referencing new row as clause', 'referencing new table as clause', 'referencing old row as clause', 'referencing old table as clause', 'on delete cascade', 'on update cascade', 'cascades', 'foreign keys'
    Relational Model & Algebra: 'relation', 'relational model', 'relational schema', 'relational instance', 'relational algebra', 'relational-algebra expressions', 'functional dependencies', 'multiset relational algebra', 'Multisets', 'multiset except', 'Set comparisons in SQL', 'compatible relations', 'equivalence', 'equivalent queries', 'Conceptual evaluation strategy', 'monotonic queries', 'binary operations', 'unary operations', 'rename operation'
    Row-Level Security: 'Row-level triggers'
    SQL Standards & History: 'Structured Query Language (SQL)', 'Sequel', 'American National Standards Institute (ANSI)', 'International Organization for Standardization (ISO)', 'standards', 'SQL environment', 'conformance levels'
    Scalar Functions: 'cast', 'coalesce function', 'every function'
    Schema: 'create schema', 'drop schema', 'schemas', 'schema diagrams'
    Security: 'SQL injection'
    Select Variants: 'SELECT', 'select clause', 'select distinct', 'select all', 'select operation', 'select privilege', 'select authorization, privileges and', 'select-from-where', 'Selection', 'base query', 'restriction'
    Sequence: 'create sequence construct'
    Set & Assignment: 'set clause', 'Set', 'assignment operation', 'set null', 'set statement'
    Set Operations: 'Union', 'union all', 'union of sets', 'intersect all', 'Intersection', 'outer union operation', 'set operations', 'Set operators'
    Statistics: 'histograms'
    String Functions: 'Strings in SQL', 'string operations', 'trim', 'LIKE', 'escape', 'Escape character', 'Case sensitivity', 'Collations in SQL'
    Subqueries: 'Subquery', 'Nested queries', 'nested subqueries', 'correlated subqueries', 'scalar subqueries'
    Table: 'create table...as', 'create table...like', 'create temporary table', 'alter table', 'drop table', 'tables'
    Table Functions: 'table functions'
    Temporal Concepts: 'as of period for', 'versions period for', 'period declaration', 'valid time', 'temporal validity', 'current date', 'localtimestamp', 'Dates and times in SQL', 'timezone', 'timestamp'
    Transactions & Isolation: 'Transaction', 'Commit', 'rollback', 'rollback work', 'automatic commit', 'set autocommit off', 'transaction control', 'Read commited', 'Read uncommited', 'Repeatable read', 'Serializability', 'Isolation level', 'Read-only transaction'
    Triggers: 'triggers', 'after triggers', 'before triggers', 'Row-level triggers', 'Statement-level triggers', 'Events activating triggers', 'CREATE TRIGGER', 'alter trigger', 'disable trigger', 'drop trigger', 'for each row clause', 'for each statement clause', 'instead of feature', 'transition tables', 'transition variables'
    Type: 'distinct type', 'create distinct type', 'CREATE TYPE', 'alter type', 'drop type', 'types', 'user-defined types', 'structured types'
    View: 'views', 'create view', 'parameterized views', 'create recursive view', 'view definition', 'materialized views', 'view maintenance'
    WITH Clauses: 'with clause', 'with data clause', 'with check option', 'with grant option', 'with timezone specification'
    Windowing & Pivoting: 'windows and windowing', 'pivot clause', 'pivot attribute', 'pivot-table', 'pivoting', 'ranking'
'''
category_terms = defaultdict(list)
for line in text.strip().splitlines():
    match = re.match(r"\s*(.*?):\s*(.*)", line)
    if match:
        category = match.group(1).strip()
        terms = re.findall(r"'(.*?)'", match.group(2))
        category_terms[category].extend([term.strip() for term in terms])

# Extract all terms and remove duplicates
all_terms = set()
for terms in category_terms.values():
    all_terms.update(terms)

# Print the number of categories
print(f"There are {len(category_terms)} categories.")

# If you want to see all category names and some examples:
print("Category examples:", list(category_terms.keys())[:5])  # First 5 categories
print("Total number of unique terms:", len(all_terms))

There are 70 categories.
Category examples: ['Active Databases', 'Aggregate Functions', 'Aliases & Correlation', 'Arity', 'Atomicity & Domains']
Total number of unique terms: 406


# Valid Classification Statistics

In [3]:
import json
from collections import Counter

# modify this to your own file path
input_file = 'sql_subtopics_classification/sql_subtopics_classification_results_qwen3.jsonl'

# Initialize counter
category_counter = Counter()

# Read file and count
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        matching_categories = data.get("matching_categories", [])
        # Count categories that are in category_terms
        for category in matching_categories:
            if category in category_terms:
                category_counter[category] += 1

# Output results
for category, freq in category_counter.items():
    print(f"{category}: {freq}")

# Output number of categories
print(len(category_counter), "categories found in the data.")
# Categories not in the data
missing_categories = set(category_terms.keys()) - set(category_counter.keys())
print("Missing categories:", missing_categories)
# Total frequency of all categories
total_frequency = sum(category_counter.values())
print("Total frequency of all categories:", total_frequency)


Join Operations: 717
Relational Model & Algebra: 134
Projection & Project Operation: 9
Select Variants: 303
Data Manipulation Language (DML): 1335
Expressions & Syntax: 882
Queries & Paradigms: 224
Database Systems: 150
Transactions & Isolation: 265
Index: 61
Data Types - Scalar: 129
Group BY & Having: 682
Data Definition Language (DDL): 326
Aggregate Functions: 582
Logical Connectives: 268
String Functions: 88
Authorization & Privileges: 10
Partitioning: 5
SQL Standards & History: 46
Schema: 96
Key Constraints: 62
Domain & Check Constraints: 57
Procedures & PSM: 71
Windowing & Pivoting: 193
Duplicate Handling: 95
Table: 128
Integrity Constraints: 53
Data Types - Large Objects: 39
Subqueries: 650
Ordering & Limits: 410
Null & Unknown Handling: 44
Referential Integrity: 50
Security: 48
Backup & Recovery: 27
Catalogs & Metadata: 5
Triggers: 42
Recursive Queries: 90
Scalar Functions: 53
Set Operations: 36
Set & Assignment: 3
Cartesian & Product: 8
Active Databases: 3
Aliases & Correlation

In [4]:
print(len(category_counter), "categories found in the data.")

61 categories found in the data.


In [8]:
import json
import sqlite3
from collections import defaultdict, Counter
import statistics

# File and database paths
jsonl_file = 'sql_subtopics_classification/sql_subtopics_classification_results_qwen3.jsonl'
db_file = 'ds_edu_videos.db'

# Collect video IDs for each category and count videos
category_videos = defaultdict(list)
category_counter = Counter()

with open(jsonl_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        video_id = data.get('video_id')
        matching_categories = data.get('matching_categories', [])
        for category in matching_categories:
            if category in category_terms:
                category_videos[category].append(video_id)
                category_counter[category] += 1

# Connect to database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Store statistics results
result = {}

for category, video_ids in category_videos.items():
    views = []
    likes = []
    comments = []
    durations = []
    
    for vid in video_ids:
        cursor.execute('SELECT view_count, like_count, comment_count, duration FROM videos WHERE video_id = ?', (vid,))
        row = cursor.fetchone()
        if row:
            view_count, like_count, comment_count, duration = row
            # Safe conversion
            try:
                if view_count is not None:
                    views.append(int(view_count))
            except:
                pass
            try:
                if like_count is not None:
                    likes.append(int(like_count))
            except:
                pass
            try:
                if comment_count is not None:
                    comments.append(int(comment_count))
            except:
                pass
            if duration:
                parts = duration.split(':')
                try:
                    if len(parts) == 3:
                        h, m, s = map(int, parts)
                        total_minutes = h * 60 + m + s / 60
                        durations.append(total_minutes)
                    elif len(parts) == 2:
                        m, s = map(int, parts)
                        total_minutes = m + s / 60
                        durations.append(total_minutes)
                except:
                    pass
    
    result[category] = {
        'video_count': category_counter[category],
        'median_view': statistics.median(views) if views else None,
        'median_like': statistics.median(likes) if likes else None,
        'median_comment': statistics.median(comments) if comments else None,
        'median_duration_min': statistics.median(durations) if durations else None
    }

conn.close()

# Sort by video count and print
sorted_results = sorted(result.items(), key=lambda x: x[1]['video_count'], reverse=True)

for category, stats in sorted_results:
    print(f"Category: {category}")
    print(f"  Video Count: {stats['video_count']}")
    print(f"  Median View Count: {stats['median_view']}")
    print(f"  Median Like Count: {stats['median_like']}")
    print(f"  Median Comment Count: {stats['median_comment']}")
    print(f"  Median Duration (min): {stats['median_duration_min']}")
    print()


Category: Data Manipulation Language (DML)
  Video Count: 1335
  Median View Count: 5011
  Median Like Count: 78.5
  Median Comment Count: 5
  Median Duration (min): 8.75

Category: Expressions & Syntax
  Video Count: 882
  Median View Count: 4883.0
  Median Like Count: 101.5
  Median Comment Count: 5.0
  Median Duration (min): 12.125

Category: Join Operations
  Video Count: 717
  Median View Count: 8509
  Median Like Count: 188
  Median Comment Count: 10.0
  Median Duration (min): 12.016666666666667

Category: Group BY & Having
  Video Count: 682
  Median View Count: 1986.0
  Median Like Count: 35.5
  Median Comment Count: 3
  Median Duration (min): 10.458333333333334

Category: Subqueries
  Video Count: 650
  Median View Count: 5481.0
  Median Like Count: 89
  Median Comment Count: 6
  Median Duration (min): 11.666666666666666

Category: Aggregate Functions
  Video Count: 582
  Median View Count: 666.5
  Median Like Count: 14
  Median Comment Count: 1.0
  Median Duration (min): 10.2

# Missing Categories Statistics

In [3]:
# categories in matching_categories but not in category_terms
import json
from collections import Counter

input_file = 'sql_subtopics_classification/sql_subtopics_classification_results_qwen3.jsonl'

# categories that appear in matching_categories **but not** in category_terms
missing_counter = Counter()

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        matching_categories = data.get("matching_categories", [])

        # count every occurrence (including duplicates across the file)
        for cat in matching_categories:
            if cat not in category_terms: 
                missing_counter[cat] += 1

# pretty-print the result
for cat, freq in missing_counter.items():
    print(f"{cat}: {freq}")

# number of categories found in missing_counter
print(len(missing_counter), "categories found in the data.")
# total frequency of all missing categories
total_missing_freq = sum(missing_counter.values())
print("Total frequency of all missing categories:", total_missing_freq)

Performance Optimization: 5
Performance Tuning: 2
Optimization: 3
Data Control Language (DCL): 13
Transaction Control Language (TCL): 8
Constraints: 10
Joins: 3
Views: 55
Indexes: 2
Functions: 2
Execution Plans: 3
Data Manipulation Language (DML) - Insertion: 1
Data Manipulation Language (DML) - Update: 1
Data Manipulation Language (DML) - Deletion: 1
Data Manipulation Language (DML) - Subqueries: 1
Numeric Functions: 1
Date and Time Functions: 1
Installation & Setup: 7
Constraints & Validation: 3
Stored Procedures: 6
Insertion: 240
Update: 102
Delete: 57
Date & Time Functions: 6
Duplicates Handling: 12
Normalization: 6
Data Control Language: 1
Constraints & Integrity: 6
Joins & Join Operations: 1
Deletion: 24
Primary Key & Foreign Key Constraints: 1
Tables: 4
Docker: 1
Primary & Foreign Keys: 1
Stored Procedures & PSM: 1
Alter Table: 8
Constraints & Check: 1
Date Functions: 1
Distinct & Duplicate Handling: 9
Primary Key: 1
Unique Key: 1
Cluster Indexes: 1
Non-Cluster Indexes: 1
CTE: 2

In [ ]:
# 10 videos with no categories
with open(input_file, 'r', encoding='utf-8') as f:
    no_category_videos = [json.loads(line) for line in f if not json.loads(line).get("matching_categories")]
# Print the first 10 videos with no categories
for video in no_category_videos[:10]:
    print(video.get("video_id"), video.get("title"), video.get("description"))

gTK5rNhWJyA None None
1yktzZFhx2Q None None
tk5SB8jiK7U None None
-bt_y4Loofg None None
J6mDkcqU_ZE None None
sQD7kaZ5h0s None None
ZCc6pzQzCIA None None
pKzazzwVEDg None None
On9eSN3F8w0 None None
GUP3Z2PsktU None None


In [5]:
import json
from collections import Counter

input_file = 'sql_subtopics_classification/sql_subtopics_classification_results_qwen3.jsonl'

missing_counter = Counter()
no_category_count = 0
only_invalid_category_count = 0
total_records = 0

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        total_records += 1
        data = json.loads(line)
        matching_categories = data.get("matching_categories", [])

        if not matching_categories:
            # No category at all
            no_category_count += 1
        else:
            # Check if all are invalid categories
            invalid_cats = [cat for cat in matching_categories if cat not in category_terms]
            
            if len(invalid_cats) == len(matching_categories):
                only_invalid_category_count += 1

            # Count frequency of each invalid category
            for cat in invalid_cats:
                missing_counter[cat] += 1

# Output results
print(f"Number of videos with no category: {no_category_count}")
print(f"Number of videos with only invalid categories: {only_invalid_category_count}")
# Video IDs with only invalid categories
only_invalid_video_ids = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        matching_categories = data.get("matching_categories", [])
        if matching_categories and all(cat not in category_terms for cat in matching_categories):
            only_invalid_video_ids.append(data.get("video_id"))
print(f"Video IDs with only invalid categories: {only_invalid_video_ids}")
print(f"Total number of videos: {total_records}")
print()

print("Frequency of each invalid category:")
for cat, freq in missing_counter.items():
    print(f"{cat}: {freq}")

print()
print(f"Total number of invalid category types: {len(missing_counter)}")
print(f"Total occurrences of all invalid categories: {sum(missing_counter.values())}")


Number of videos with no category: 265
Number of videos with only invalid categories: 63
Video IDs with only invalid categories: ['xAZrlhpqw54', 'EXA6ZmgZ0Kw', 'PHsC_t0j1dU', 'AZ29DXaJ1Ts', 'pO31_tTKByA', 'HeqmOAryf84', 'FHKtelZpa4Q', 'ko_ls0E7xSU', 'RrjfLNzyklA', 'MPaw_lWcyuo', 'jbwHpQKPHqw', 'WpQECq5Hx9g', 'idFCAFAgOO0', 'bmc-yBcWMw8', 'oEUhNCQOWuw', 'j4BnAgqY90g', 'uDEXDDGr_Bs', '5rLbBpu1f6E', 'fhilmNQPmnU', 'n0gl-pXG9Z4', 'qUXnu9dpur4', 'U8mlVeJxhLk', 'wUByBj6yIoA', 'nnvObiq3Z5A', 't2dMQw_yuoc', 'K1WeoKxLZ5o', 'FwcAkH8UyEA', 'ZnBaA6FO9BI', 'pIpylD5jBEg', '3QuYuU9OJfA', 'S1jw7DpEGGU', 'bQYPb44RU9I', 'F2Wo0mHcXzM', 'LMzSo9GsLNs', 'yfDM44FuzoU', 'cNZIs-Q8H4U', 'Vi5YRegJuFs', 'x8ds2QVsXNc', '4QuT6OwZ9ic', 'mF1At1iytbw', 'teUcMPYpR5k', 'dOzSirQMoiw', 't5HcdVXoD-I', 'fROYku0EyS0', 'D45mzmBNnJc', 'yzS10bUcAhU', 'fN2zDE-1p2Q', 'jj1c49iKXfg', '2lGg41PAhVU', '9xM7pkYayeM', 'L4EuuFt8nqQ', 'Hcz2ZjO8UiA', '2fbtqiUQKqE', 'X2A9trAe7x0', 'VOvwXPYjhvk', 'HL8PM3QijNs', '3rZZt9lL-po', 'mkn5_g4bJl0', 

In [13]:
# categories not in the data
missing_categories = set(category_terms.keys()) - set(category_counter.keys())
print("Missing categories:", missing_categories)

Missing categories: {'Arity', 'Operating Systems', 'Pointers', 'Exceptions & Debugging', 'Atomicity & Domains', 'Difference & EXCEPT', 'Row-Level Security', 'Example Databases', 'Change Tracking & Delta'}
